In [13]:
import pandas as pd
import matplotlib.pyplot as plt

In [6]:
df = pd.read_csv("../RawDataDB.csv")

In [7]:
df.describe(include="all")

,manufacturer,model,mileage,year,fuel,price,currency,city,county,status,short_description,long_description
count,36534,36534,36534,36534,36534,36534,36534,36534,36533,36534,36506,36534
unique,64,1078,12842,1250,396,6862,2,686,126,2,9676,30012
top,Mercedes-Benz,Passat,Diesel,2019,Diesel,7 990,EUR,Bucuresti,(Bucuresti,Publicat,[],1 968 cm3 • 150 CP
freq,4857,1144,1064,4031,21444,178,36499,8696,12848,20101,9130,216


In [8]:
df.head()

,manufacturer,model,mileage,year,fuel,price,currency,city,county,status,short_description,long_description
0,Abarth,595,49 280 km,2020,Benzina,19 990,EUR,Bucuresti,(Bucuresti,Reactualizat,[],1 368 cm3 • 180 CP
1,Abarth,500,100 400 km,2009,Benzina,7 900,EUR,Cluj-Napoca,(Cluj),Reactualizat,[],1 368 cm3 • 136 CP • Sport / Clima / Incalzire...
2,Abarth,595,49 300 km,2020,Benzina,18 988,EUR,Ploiesti,(Prahova),Publicat,[],"1 368 cm3 • 180 CP • TVA inclus,deductibil /Pr..."
3,Abarth,595,53 000 km,2016,Benzina,16 290,EUR,Brasov,(Brasov),Reactualizat,[],1 368 cm3 • 180 CP • Fiat 500 Abarth/ 595 Comp...
4,Abarth,595,29 330 km,2020,Benzina,18 990,EUR,Bucuresti,(Bucuresti,Reactualizat,1.4 T-Jet,1 368 cm3 • 145 CP


In [9]:
df.tail()

,manufacturer,model,mileage,year,fuel,price,currency,city,county,status,short_description,long_description
36529,Volvo,V40 D2,201 000 km,2013,Diesel,6 500,EUR,Cluj-Napoca,(Cluj),Publicat,ECO Kinetic,"1 560 cm3 • 115 CP • V40, Diesel 1.6 euro-5, J..."
36530,Volvo,S60 D4,119 406 km,2015,Diesel,14 500,EUR,Drobeta-Turnu,Severin,Publicat,Start-Stop Kinetic,"1 984 cm3 • 163 CP • PROPRIETAR, vand VOLVO S6..."
36531,Volvo,V40 D2,241 950 km,2013,Diesel,7 950,EUR,Pitesti,(Arges),Publicat,ECO Summum,1 560 cm3 • 115 CP • 2013 / Summum / 1.6d 115 ...
36532,Volvo,XC 60,184 000 km,2021,Hibrid,33 900,EUR,Miercurea-Ciuc,(Harghita),Publicat,T8 Twin Engine AWD Geartronic Inscription,1 969 cm3 • 303 CP • Volvo XC 60 T8 Twin Engin...
36533,Volvo,XC 90,361 000 km,2004,Diesel,4 100,EUR,Eforie,Nord,Publicat,2.4 D Aut. 7 locuri,2 401 cm3 • 163 CP • Proprietar de 3 ani. Star...


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36534 entries, 0 to 36533
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   manufacturer       36534 non-null  object
 1   model              36534 non-null  object
 2   mileage            36534 non-null  object
 3   year               36534 non-null  object
 4   fuel               36534 non-null  object
 5   price              36534 non-null  object
 6   currency           36534 non-null  object
 7   city               36534 non-null  object
 8   county             36533 non-null  object
 9   status             36534 non-null  object
 10  short_description  36506 non-null  object
 11  long_description   36534 non-null  object
dtypes: object(12)
memory usage: 3.3+ MB
